In [ ]:
# 04_eval.ipynb: Model Evaluation & Visualization

# This notebook loads cross-validation results, computes evaluation metrics, and generates key figures for the report.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from joblib import load
from sklearn.model_selection import train_test_split
from src.preprocess import build_pipeline
from src.models import svm

if Path("../reports/results_table2.csv").exists():
    results = pd.read_csv("../reports/results_table2.csv")
elif Path("../results_table2.csv").exists():
    results = pd.read_csv("../results_table2.csv")
else:
    raise FileNotFoundError("results_table2.csv not found in expected locations.")

print(results.head())

# results = pd.read_csv("../reports/results_table2.csv")
# print(results.head())


# Adjust file path as needed
#results = pd.read_csv("results_table2.csv")  # Or the CSV generated by gather_results.py
#print(results.head())


In [ ]:
# Example: Plot confusion matrix for the best pipeline
y_true = pd.read_csv("../data/test_labels.csv")['label_name']  # Load your true labels (from held-out data/split)
y_pred = pd.read_csv("../reports/best_model_predictions.csv")['pred_label']   # Load predictions from the best model

cm = confusion_matrix(y_true, y_pred, labels=sorted(set(y_true)))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=sorted(set(y_true)))
disp.plot(cmap="Blues", xticks_rotation=45)
plt.title("Confusion Matrix (Best Model)")
plt.tight_layout()
plt.savefig("../reports/figures/cm_best.png")
plt.show()


# cm = confusion_matrix(y_true, y_pred, labels=sorted(set(y_true)))
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=sorted(set(y_true)))
# disp.plot(cmap="Blues", xticks_rotation=45)
# plt.title("Confusion Matrix (Best Model)")
# plt.savefig("../reports/figures/cm_best.png")
# plt.show()

In [ ]:
# Example: Plot ROC curve (multi-class, one-vs-rest)
# You may need to binarize y for multiclass
# For each class:
# fpr, tpr, _ = roc_curve(y_true_binary, y_score_for_this_class)
# plt.plot(fpr, tpr, label=f'Class {i}')
plt.plot([0, 1], [0, 1], 'k--')
plt.title("ROC Curve (Best Model)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.savefig("../reports/figures/roc_best.png")
plt.show()

In [ ]:
# Load your cleaned data

df = pd.read_feather("../data/tweets.feather")
X = df['text']
y = df['label_name']

# Use stratified split for reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [ ]:
# Build your best pipeline

pipe = build_pipeline("../conf/rep_word_tfidf.yaml")
pipe.steps.append(("clf", svm()))
pipe.fit(X_train, y_train)


In [ ]:
# Generate test predictions

y_pred = pipe.predict(X_test)


In [ ]:
# Load saved pipeline

# Load your saved pipeline (adjust the path and filename)
pipe = load("../reports/best_model.joblib")
X_test = pd.read_feather("../data/test.feather")['text']
y_true = pd.read_feather("../data/test.feather")['label_name']
y_pred = pipe.predict(X_test)


In [ ]:
# Save test predictions

pd.DataFrame({
    "text": X_test,
    "true_label": y_test,
    "pred_label": y_pred
}).to_csv("../reports/best_model_predictions.csv", index=False)


In [ ]:
# confusion matrix plotting

y_true = y_test
# Or, if loading from CSV:
# y_true = pd.read_csv("../reports/best_model_predictions.csv")["true_label"]
# y_pred = pd.read_csv("../reports/best_model_predictions.csv")["pred_label"]

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_true, y_pred, labels=sorted(set(y_true)))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=sorted(set(y_true)))
disp.plot(cmap="Blues", xticks_rotation=45)
plt.title("Confusion Matrix (Best Model)")
plt.tight_layout()
plt.savefig("../reports/figures/cm_best.png")
plt.show()
